In [1]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/332.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/332.0 MB 1.3 MB/s eta 0:04:14
   ---------------------------------------- 0.8/332.0 MB 1.6 MB/s eta 0:03:29
   ---------------------------------------- 2.4/332.0 MB 3.1 MB/s eta 0:01:48
   ---------------------------------------- 3.1/332.0 MB 3.3 MB/s eta 0:01:40
    --------------------------------------- 4.2/332.0 MB 3.5 MB/s eta 0:01:35
    --------------------------------------- 4.7/332.0 MB 3.6 MB/s eta 0:01:32
    --------------------------------------- 5.5/332.0 MB 3.4 MB/s eta 0:01:37
    --------------------------------------- 6.6/332.0 MB 3.6 MB/s eta 0:01:32
    --------------------------------------- 6.8/332.0 MB 3.5 MB/s eta 0:01:34
    ---

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

In [ ]:
#Class Task: Build a simple ANN.

In [2]:
# Activation Function (Sigmoid)
def sigmoid(x):
    """The sigmoid activation function."""
    return 1 / (1 + np.exp(-x))

# 2. Define Derivative of Sigmoid
def sigmoid_derivative(x):
    """The derivative of the sigmoid function."""
    return x * (1 - x)

# Input and Output for XOR
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) # Input
y_xor = np.array([[0], [1], [1], [0]])            # Expected Output

# Neural Network Architecture Parameters
input_size = 2
hidden_size = 4
output_size = 1
learning_rate = 0.1
epochs = 10000

# Initialize Weights (Randomly)
np.random.seed(42) # For reproducible results
# Weights between input layer (l0) and hidden layer (l1)
weights_l0_l1 = np.random.uniform(size=(input_size, hidden_size))
# Weights between hidden layer (l1) and output layer (l2)
weights_l1_l2 = np.random.uniform(size=(hidden_size, output_size))

# --- Training Loop ---
for epoch in range(epochs):

    # Forward Propagation
    # Layer 1 (Hidden Layer)
    l1_input = np.dot(X_xor, weights_l0_l1)
    l1_output = sigmoid(l1_input)

    # Layer 2 (Output Layer)
    l2_input = np.dot(l1_output, weights_l1_l2)
    l2_output = sigmoid(l2_input) # Final Prediction (h)

    # Calculate Loss (Error)
    error = y_xor - l2_output

    # Backpropagation 

    # Output Layer Error Gradient (d_loss/d_weights_l1_l2)
    l2_delta = error * sigmoid_derivative(l2_output)

    # Hidden Layer Error Contribution (propagating error backwards)
    l1_error = l2_delta.dot(weights_l1_l2.T)
    l1_delta = l1_error * sigmoid_derivative(l1_output)

    # Weight Update (Gradient Descent)
    
    # Update weights_l1_l2
    weights_l1_l2 += l1_output.T.dot(l2_delta) * learning_rate
    # Update weights_l0_l1
    weights_l0_l1 += X_xor.T.dot(l1_delta) * learning_rate

    # Print loss every 1000 epochs
    if epoch % 1000 == 0:
        mse = np.mean(np.square(error))
        print(f"Epoch {epoch:5d} Loss (MSE): {mse:.6f}")

# --- Testing the Trained Model ---
l1_final = sigmoid(np.dot(X_xor, weights_l0_l1))
l2_final = sigmoid(np.dot(l1_final, weights_l1_l2))

print("\n--- XOR Prediction Results (After Training) ---")
print("Input (X_xor):\n", X_xor)
print("Predicted Output (l2_final):\n", l2_final.round(3))
print("Target Output (y_xor):\n", y_xor)


Epoch     0 Loss (MSE): 0.341657
Epoch  1000 Loss (MSE): 0.245261
Epoch  2000 Loss (MSE): 0.209176
Epoch  3000 Loss (MSE): 0.151009
Epoch  4000 Loss (MSE): 0.086110
Epoch  5000 Loss (MSE): 0.035729
Epoch  6000 Loss (MSE): 0.019402
Epoch  7000 Loss (MSE): 0.012958
Epoch  8000 Loss (MSE): 0.009662
Epoch  9000 Loss (MSE): 0.007687

--- XOR Prediction Results (After Training) ---
Input (X_xor):
 [[0 0]
 [0 1]
 [1 0]
 [1 1]]
Predicted Output (l2_final):
 [[0.101]
 [0.927]
 [0.92 ]
 [0.059]]
Target Output (y_xor):
 [[0]
 [1]
 [1]
 [0]]


In [ ]:
# Assignment: 
1-Apply ANN on your dataset.
2-Compare results with earlier models.

In [4]:
NUM_CUSTOMERS = 1000
NUM_ITEMS = 500
TOTAL_TRANSACTIONS = 50000

# Create realistic non-uniform data
customer_ids = np.random.choice(range(10000, 10000 + NUM_CUSTOMERS), size=TOTAL_TRANSACTIONS)
item_stock_codes = np.random.choice([f'ITEM_{i:04d}' for i in range(NUM_ITEMS)], size=TOTAL_TRANSACTIONS)
# Simulate Quantity purchased (often skewed, mostly small numbers)
quantities = np.random.poisson(lam=2, size=TOTAL_TRANSACTIONS) + 1 # Min 1

raw_data = pd.DataFrame({
    'CustomerID': customer_ids, # Corresponds to Customer ID (User)
    'StockCode': item_stock_codes, # Corresponds to StockCode (Item)
    'Quantity': quantities # Corresponds to Quantity (used to derive preference score)
})

# Aggregate to get the total purchase count for each Customer-Item pair (our preference proxy)
# This step is crucial for transforming transaction-level data into user-item preference data.
df = raw_data.groupby(['CustomerID', 'StockCode'])['Quantity'].sum().reset_index()
df.rename(columns={'Quantity': 'purchase_count'}, inplace=True)

print(f"Simulated dataset shape (Aggregated User-Item Preferences): {df.shape}")
print("Sample Data (CustomerID, StockCode, purchase_count):")
print(df.head())

Simulated dataset shape (Aggregated User-Item Preferences): (47602, 3)
Sample Data (CustomerID, StockCode, purchase_count):
   CustomerID  StockCode  purchase_count
0       10000  ITEM_0018               4
1       10000  ITEM_0043               5
2       10000  ITEM_0061               8
3       10000  ITEM_0069               2
4       10000  ITEM_0073               3


In [5]:
# 2. Preprocessing Steps (MANDATORY for ANNs in Recommenders)

# A. Encoding/Mapping User and Item IDs
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

# Map original IDs (Customer ID and StockCode) to sequential integers
df['user_encoded'] = user_encoder.fit_transform(df['CustomerID'])
df['item_encoded'] = item_encoder.fit_transform(df['StockCode'])

# Determine vocabulary sizes for embedding layers
num_users = df['user_encoded'].nunique()
num_items = df['item_encoded'].nunique()
print(f"\nTotal Customers: {num_users}, Total Items: {num_items}")

# B. Scaling the Target (Purchase Count / Preference Score)
# Scaling the target (purchase_count) to a range (0 to 1) for regression.
rating_scaler = MinMaxScaler(feature_range=(0, 1))
df['rating_scaled'] = rating_scaler.fit_transform(df[['purchase_count']])



Total Customers: 1000, Total Items: 500


In [7]:
# 3. Prepare Data for Model Training
X = df[['user_encoded', 'item_encoded']]
y = df['rating_scaled']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

train_user_ids = X_train['user_encoded']
train_item_ids = X_train['item_encoded']
test_user_ids = X_test['user_encoded']
test_item_ids = X_test['item_encoded']

In [9]:
# 4. Build the ANN Model (Collaborative Filtering Approach)

# We use two separate input branches (for user and item) merged into a single network.
# Embedding Size: Rule of thumb is often (Vocab Size)**0.25 to 50. We use 20 here.
embedding_size = 20

# A. User Input Branch
user_input = Input(shape=(1,), name='User_Input')
# Embedding layer learns a dense vector representation (features) for each user
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_size,
                           name='User_Embedding')(user_input)
user_flatten = Flatten()(user_embedding)

# B. Item Input Branch
item_input = Input(shape=(1,), name='Item_Input')
# Embedding layer learns a dense vector representation (features) for each item
item_embedding = Embedding(input_dim=num_items, output_dim=embedding_size,
                           name='Item_Embedding')(item_input)
item_flatten = Flatten()(item_embedding)

# C. Concatenate and Dense Layers
# Merge the user and item feature vectors
merged_vector = Concatenate()([user_flatten, item_flatten])

# Apply dense layers to learn complex interaction patterns
dense_layer_1 = Dense(256, activation='relu')(merged_vector) # Increased complexity
dense_layer_2 = Dense(128, activation='relu')(dense_layer_1)
dense_layer_3 = Dense(64, activation='relu')(dense_layer_2)

# Output layer: Single node, sigmoid activation for prediction between 0 and 1
# (Matches our scaled rating target)
output_layer = Dense(1, activation='sigmoid', name='Prediction')(dense_layer_3)

# Create the final model
model = Model(inputs=[user_input, item_input], outputs=output_layer)


In [10]:
# 5. Compile and Train the Model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=MeanSquaredError(), # Regression task (predicting a scaled purchase count)
    metrics=['mae'] # Mean Absolute Error for easier interpretation
)

print("\nModel Summary:")
model.summary()

history = model.fit(
    [train_user_ids, train_item_ids], y_train,
    batch_size=128, # Increased batch size
    epochs=15, # Increased epochs
    verbose=1,
    validation_data=([test_user_ids, test_item_ids], y_test)
)


Model Summary:


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ User_Input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Item_Input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ User_Embedding (Embedding)    │ (None, 1, 20)             │          20,000 │ User_Input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Item_Embedding (Embedding)    │ (None, 1, 20)             │          10,000 │ Item_Input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_4 (Flatten)           │ (None, 20)                │               0 │ User_Embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_5 (Flatten)           │ (None, 20)                │               0 │ Item_Embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_2 (Concatenate)   │ (None, 40)                │               0 │ flatten_4[0][0],           │
│                               │                           │                 │ flatten_5[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_6 (Dense)               │ (None, 256)               │          10,496 │ concatenate_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_7 (Dense)               │ (None, 128)               │          32,896 │ dense_6[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_8 (Dense)               │ (None, 64)                │           8,256 │ dense_7[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Prediction (Dense)            │ (None, 1)                 │              65 │ dense_8[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 81,713 (319.19 KB)

 Trainable params: 81,713 (319.19 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
298/298 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0167 - mae: 0.0953 - val_loss: 0.0112 - val_mae: 0.0815
Epoch 2/15
298/298 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0112 - mae: 0.0816 - val_loss: 0.0114 - val_mae: 0.0834
Epoch 3/15
298/298 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0111 - mae: 0.0813 - val_loss: 0.0114 - val_mae: 0.0835
Epoch 4/15
298/298 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0108 - mae: 0.0804 - val_loss: 0.0117 - val_mae: 0.0833
Epoch 5/15
298/298 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0102 - mae: 0.0783 - val_loss: 0.0123 - val_mae: 0.0869
Epoch 6/15
298/298 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0093 - mae: 0.0752 - val_loss: 0.0133 - val_mae: 0.0917
Epoch 7/15
298/298 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0083 - mae: 0.0714 - val_loss: 0.0140 - val_mae: 0.0924
Epoch 8/15
298/298 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0074 - mae: 0.0678 - val_loss: 0.0145 - val_mae: 0.0926
Epoch 9/15
298/298 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - lo

In [12]:
# 6. Evaluate and Predict
loss, mae = model.evaluate([test_user_ids, test_item_ids], y_test, verbose=0)
print(f"\nTest Mean Absolute Error (MAE): {mae:.4f}")

# Example Prediction: Predict a random user/item pair preference
# Select a known user and a known item from the test set for prediction
example_idx = X_test.index[0]
user_to_predict_encoded = X_test.loc[example_idx, 'user_encoded']
item_to_predict_encoded = X_test.loc[example_idx, 'item_encoded']

# FIX: Convert single encoded IDs into NumPy arrays of shape (1,) for robust batch prediction
user_input_array = np.array([user_to_predict_encoded])
item_input_array = np.array([item_to_predict_encoded])

# Predict using the NumPy arrays
sample_prediction_scaled = model.predict([user_input_array, item_input_array], verbose=0)[0][0]

sample_prediction_original = rating_scaler.inverse_transform([[sample_prediction_scaled]])[0][0]

# Find the original user and item IDs for printing
user_to_predict_id = user_encoder.inverse_transform([user_to_predict_encoded])[0]
item_to_predict_id = item_encoder.inverse_transform([item_to_predict_encoded])[0]

print(f"\n--- Example Prediction ---")
print(f"Customer ID: {user_to_predict_id}, StockCode: {item_to_predict_id}")
print(f"Predicted Scaled Preference (0-1): {sample_prediction_scaled:.4f}")
print(f"Predicted Purchase Count (Original Scale): {sample_prediction_original:.2f}")


Test Mean Absolute Error (MAE): 0.1062

--- Example Prediction ---
Customer ID: 10680, StockCode: ITEM_0478
Predicted Scaled Preference (0-1): 0.1216
Predicted Purchase Count (Original Scale): 2.82


In [13]:
# Comparison with earlier models (Conceptual)
print("\nANN Performance Analysis:")
print("1. Simple ANN (NumPy):")
print("   - Successfully converged to solve the non-linear XOR problem.")
print("   - Requires manual calculation of gradients and high maintenance.")
print("   - Low-level, excellent for understanding core mechanics (Forward/Backpropagation).")

print("\n2. E-commerce Recommendation ANN (Keras - Using Purchase Count):")
print(f"   - MAE of {mae:.4f} indicates the model's predicted scaled preference is off by that amount on average.")
print("   - The model is now designed to predict a scaled version of 'Purchase Count' (quantity aggregated per user-item pair), which serves as a better proxy for customer preference in this dataset.")
print("   - The original 'rating' scale is the range of 'Purchase Count' (e.g., 1 to max quantity), making this a robust regression task.")
print("   - We increased model complexity (embedding size and dense layers) to handle the more realistic simulated scale and sparsity of the e-commerce data.")
print("   - We have removed the deprecated 'input_length' parameter from the Embedding layers to clear a warning.")

# Conceptual Comparison with Traditional Models (e.g., Linear Regression, K-NN, Matrix Factorization)
print("\nConceptual Comparison:")
print("   - **Traditional Models (e.g., Linear Regression or simpler Matrix Factorization):** These models are faster to train and more interpretable. However, they struggle to capture complex, non-linear feature interactions (like the relationship between a user's purchase history and an item's description).")
print("   - **ANN (Embedding-based):** ANNs, especially those using **Embedding Layers** (as shown above), learn *latent features* for every user and item simultaneously. They are superior at capturing deep, non-linear, and complex patterns in user-item interactions, leading to much better recommendation quality (lower MAE/higher precision) given enough data and training time. The trade-off is higher computational cost and less direct interpretability.")

print("\nConclusion: The ANN model, while complex, is the industry standard for state-of-the-art recommendation systems due to its ability to model non-linear relationships, which simpler models miss.")


ANN Performance Analysis:
1. Simple ANN (NumPy):
   - Successfully converged to solve the non-linear XOR problem.
   - Requires manual calculation of gradients and high maintenance.
   - Low-level, excellent for understanding core mechanics (Forward/Backpropagation).

2. E-commerce Recommendation ANN (Keras - Using Purchase Count):
   - MAE of 0.1062 indicates the model's predicted scaled preference is off by that amount on average.
   - The model is now designed to predict a scaled version of 'Purchase Count' (quantity aggregated per user-item pair), which serves as a better proxy for customer preference in this dataset.
   - The original 'rating' scale is the range of 'Purchase Count' (e.g., 1 to max quantity), making this a robust regression task.
   - We increased model complexity (embedding size and dense layers) to handle the more realistic simulated scale and sparsity of the e-commerce data.
   - We have removed the deprecated 'input_length' parameter from the Embedding layers 